In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import *
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, accuracy_score
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

from load_data import *

In [2]:
alldf = get_df()    
# alldf = alldf.fillna(-1)
# x_train, y_train, x_pred = dg2data(alldf)

C:\Users\gango\Projects\ADA\load_data.py:14: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  alldf = pd.concat([train_df,test_df])


In [3]:
from sklearn.impute import *

In [4]:
imp_columns = ['basket_element_number', 'click_num',
       'customer_age', 'customer_value', 'duration_of_session',
       'last_order_of_customer', 'level_of_purchasing_process',
       'lifetime_customer_account', 'max_val',
       'maximum_price_of_visited_products',
       'minimum_price_of_visited_products', 'num_of_previous_payments',
       'price_of_cheapest_product_in_basket',
       'price_of_more_expensive_product_in_basket', 'regio_of_customer',
       'start_date_of_session', 'start_time_of_session',
       'sum_price_of_products_in_basket', 'sum_price_of_visited_products',
       'test_or_train_flag']

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
alldf[imp_columns] = imp.fit_transform(alldf[imp_columns])

array([[  1.        ,   1.        ,  44.76755438, ...,  49.99      ,
         49.99      ,   1.        ],
       [  1.        ,   1.        ,  44.76755438, ...,  49.99      ,
         49.99      ,   1.        ],
       [  1.        ,   1.        ,  44.76755438, ...,  49.99      ,
         49.99      ,   1.        ],
       ...,
       [  3.        ,  14.        ,  40.        , ...,  35.97      ,
        107.91      ,   0.        ],
       [  3.        ,  14.        ,  40.        , ...,  35.97      ,
        107.91      ,   0.        ],
       [  3.        ,  14.        ,  40.        , ...,  35.97      ,
        107.91      ,   0.        ]])

In [ ]:
train_df = pd.read_csv("data/public_train_trx.csv")
test_df = pd.read_csv("data/public_test_trx.csv")
alldf = pd.concat([train_df,test_df])
alldf = alldf.sort_values(['session_id','duration_of_session','click_num'])
alldf = alldf.reset_index(drop=True)
alldf['start_date_of_session'] = pd.to_datetime(alldf['start_date_of_session']).astype(np.int64)
alldf['start_time_of_session'] = pd.to_datetime(alldf['start_time_of_session']).astype(np.int64)
target = 'TARGET_successful_purchase'

alldf = alldf.fillna(-1)
# aggregalando_valtozok = list(alldf.columns)
# aggregalando_valtozok.remove('test_or_train_flag')
# aggregalando_valtozok.remove(target)
# aggregalando_valtozok.remove('session_id')

# cust_df = alldf.groupby('session_id',as_index=False).agg({target:'min',
#                                         'test_or_train_flag':'min'})
# for aggregalos_modszer in ['min','max','mean']:
#     task={}
#     ujoszlonevek=[]
#     for v in aggregalando_valtozok:
#         task[v]=aggregalos_modszer
#         ujoszlonevek.append(aggregalos_modszer+"_"+v)
#     stat = alldf.groupby(['session_id'],as_index=False).agg(task)
#     stat.columns=['session_id']+ujoszlonevek
#     cust_df = cust_df.merge(stat,on='session_id',how='left')

# bemeno_valtozok = list(cust_df.columns)[3:]
# ismert_df = cust_df[ cust_df['test_or_train_flag']==0].copy()
# x_train = ismert_df[bemeno_valtozok]
# y_train = ismert_df[target]
# x_pred = cust_df[cust_df['test_or_train_flag'] == 1][bemeno_valtozok]
# scaler = StandardScaler()
# x_train = scaler.fit_transform(x_train)
# x_pred = scaler.transform(x_pred)
# print(f"Training set size: {x_train.shape}")

In [ ]:
pd.to_datetime(alldf['start_time_of_session']).astype(np.int64)

In [ ]:
from sklearn.linear_model import LogisticRegression, RidgeClassifier, Lasso, ElasticNet, Lars, LassoLars, OrthogonalMatchingPursuit, BayesianRidge, SGDClassifier

In [ ]:
def train_model(model):
    cv_results = cross_validate(model,
                                x_train, y_train,
                                scoring=['roc_auc'], cv=5, n_jobs=4)
    auc = cv_results['test_roc_auc'].mean()
    print(f"{type(model)}: {auc}")
    return auc

In [ ]:
%%time
models = [LogisticRegression(), RidgeClassifier(), Lasso(), ElasticNet(), Lars(), LassoLars(), OrthogonalMatchingPursuit(), BayesianRidge(), SGDClassifier()]
linera_models_res = {type(model): train_model(model) for model in models}

In [ ]:
from sklearn.neighbors import *

In [ ]:
%%time
models = [KNeighborsClassifier()]
neighbor_models_res = {type(model): train_model(model) for model in models}

In [ ]:
from sklearn.tree import *

In [ ]:
%%time
models = [DecisionTreeClassifier()]
tree_models_res = {type(model): train_model(model) for model in models}

In [ ]:
all_res = {}
for res in [linera_models_res, neighbor_models_res, tree_models_res]:
    all_res.update(res)

In [ ]:
names = list(all_res.keys())
aucs = list(all_res.values())

pd.DataFrame({'name':names, 'auc': aucs}).to_csv('overview_auc.csv', index=False)

In [ ]:
param_grid = {'n_estimators': [10, 50, 100], 'max_depth': [14]}
clf = GridSearchCV(RandomForestClassifier(),
                   scoring='roc_auc',
                   param_grid=param_grid, cv=5, n_jobs=4)

print(clf.fit(x_train, y_train))

In [ ]:
best_i = np.argmax(clf.cv_results_['mean_test_score'])
print(f"Best auc: {np.max(clf.cv_results_['mean_test_score'])}")
print(f"Best params: {clf.cv_results_['params'][best_i]}")

In [ ]:
plt.imshow(clf.cv_results_['mean_test_score'])

In [ ]:
clf.cv_results_['params']

In [ ]:
pred = clf.predict_proba(x_train)[:, 1]
roc_auc_score(y_train, pred)

In [ ]:
y_train.to_numpy().astype(np.int).shape, pred.shape

In [ ]:
# aucs = []
# max_depths = []
# for max_depth in range(2,16):
#     cv_results = cross_validate(RandomForestClassifier(max_depth=max_depth),
#                                 x_train, y_train,
#                                 scoring=['roc_auc', 'accuracy'], cv=5)
#     auc = cv_results['test_roc_auc'].mean()
#     aucs.append(auc)
#     max_depths.append(max_depth)
#     print(f"Fa melyseg: {max_depth}\t{auc},\t{cv_results['test_accuracy'].mean()}")

In [ ]:
cv_results = cross_validate(DecisionTreeClassifier(max_depth = 14),
                            x_train, y_train,
                            scoring=['roc_auc', 'accuracy'], cv=5, n_jobs=-1)
auc = cv_results['test_roc_auc'].mean()
print(f"\t{auc},\t{cv_results['test_accuracy'].mean()}")

In [ ]:
# plt.plot(max_depths, aucs)
# plt.show()
# print(f"Best max_depth: {max_depths[np.argmax(aucs)]}")
# print(f"Best auc: {np.max(aucs)}")

In [ ]:
model = ExtraTreesClassifier(max_depth = 14, n_estimators = 1000)
model.fit(x_train, y_train)
pred = model.predict_proba(x_pred)
# accuracy_score(ismert_df[target],model.predict(ismert_df[bemeno_valtozok]))

In [ ]:
res_df = pd.DataFrame({'session_id': cust_df[cust_df['test_or_train_flag'] == 1].session_id, 'prob': pred[:, 1]})
res_df.to_csv('res.csv', index=False)